In [7]:
import msgpack
import ujson
import pickle
import bson
import json
#import protobuf
#import parquet

import logging
import random
import functools
import time

import cProfile, pstats, io

from fuzzer.fuzzer import DataFuzzer


logging.basicConfig(level=logging.INFO)
__fuzzer__ = logging.getLogger("fuzzer")
__profiler__ = logging.getLogger("profiler")
__timer__ = logging.getLogger("timer")

def profiler_decorator(f):

    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        t_start = None
        with cProfile.Profile() as pr:
            t_start = time.time()
            ret = f(*args, **kwargs)
        s = io.StringIO()
        sortby = "cumulative"
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        __timer__.info("Timer for {}: {} s".format(f,time.time()-t_start))
        __profiler__.info("Profiler for {}: {}".format(f,s.getvalue()))
        return ret

    return wrapper

datagenerator = DataFuzzer(_type="dict", _depth=200, _size=1024*1024)
data = datagenerator.generate_dict()
__fuzzer__.info("Generated data for testing: {}".format(data))

@profiler_decorator
def pickle_test(data):
    serialized_data = pickle.dumps(data)
    deserialized_data = pickle.loads(serialized_data)
    assert deserialized_data == data

@profiler_decorator
def ujson_test(data):
    serialized_data = ujson.dumps(data)
    deserialized_data = ujson.loads(serialized_data)
    assert deserialized_data == data

@profiler_decorator
def json_test(data):
    serialized_data = json.dumps(data)
    deserialized_data = json.loads(serialized_data)
    assert deserialized_data == data
        
@profiler_decorator
def msgpack_test(data):
    serialized_data = msgpack.packb(data, use_bin_type=True)
    deserialized_data = msgpack.unpackb(serialized_data, raw=False)
    assert deserialized_data == data

@profiler_decorator
def bson_test(data):
    serialized_data = bson.dumps(data)
    deserialized_data = bson.loads(serialized_data)
    assert deserialized_data == data

pickle_test(data)
ujson_test(data)
json_test(data)
msgpack_test(data)
bson_test(data)

#@profiler_decorator
#def parquet_test(data):
#    serialized_data = json.dumps(data)
#    deserialized_data = json.loads(serialized_data)
#    assert deserialized_data == data


INFO:fuzzer:Generated data for testing: {}


AttributeError: __enter__